In [2]:
import os

In [3]:
%pwd

'd:\\Text Summarizer Project\\TextSummarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Text Summarizer Project\\TextSummarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root]) 


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        # Extracting the data ingestion configuration from the self.config object
        config = self.config.data_ingestion
        
        # Creating necessary directories for the root directory
        create_directories([config.root_dir])
        
        # Creating the DataIngestionConfig object
        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir, source_URL=config.source_URL, local_data_file=config.local_data_file, unzip_dir=config.unzip_dir)
        
        # Returning the data ingestion configuration object
        return data_ingestion_config
            

In [9]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # If the file doesn't exist, download it
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} with following info: \n{headers}")
        else:
            # If the file exists, log its size
            file_size = os.path.getsize(Path(self.config.local_data_file))
            logger.info(f"File already exists of size: {file_size}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        Function returns None.
        """
        unzip_path = self.config.unzip_dir  # Define the unzip path
        
        # Create the unzip directory if it doesn't exist
        os.makedirs(unzip_path, exist_ok=True)
        
        # Extract the zip file
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
try:
    config  = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e: 
    raise e

[2025-01-06 09:22:38,306]: INFO: common: yaml file: config\config.yaml loaded successfully
[2025-01-06 09:22:38,317]: INFO: common: yaml file: params.yaml loaded successfully
[2025-01-06 09:22:38,328]: INFO: common: Created directory at: artifacts
[2025-01-06 09:22:38,330]: INFO: common: Created directory at: artifacts/data_ingestion
[2025-01-06 09:22:59,363]: INFO: 1574417792: Downloaded artifacts/data_ingestion/data.zip with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7182:206B06:BBAFED:116290A:677B5389
Accept-Ranges: bytes
Date: Mon, 06 Jan 2025 03:52:42 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10229-MAA
X-Cache